In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import mysql.connector
import pandas as pd
from datetime import datetime

In [9]:
# CVS 파일 읽기
df = pd.read_csv("../data/(월) 전세가격지수_아파트.csv", encoding='cp949', index_col=0)

df.reset_index(drop=True, inplace=True)
df.drop([0], axis=0, inplace=True)
df = df.transpose()
df.drop(df.index[0], inplace=True)
df.reset_index(inplace=True)
df.columns = ['date', 'type', 'national', 'seoul']

# 원자료 데이터프레임 생성
jeonse_price_index = df[df['type']=='원자료']
jeonse_price_index = jeonse_price_index.loc[:, jeonse_price_index.columns != 'type']
jeonse_price_index.reset_index(drop=True, inplace=True)
jeonse_price_index.columns = ['date', 'national_jeonse_price_index', 'seoul_jeonse_price_index']

# 전기대비증감율 데이터프레임 생성
rate_of_change = df[df['type']=='전기대비증감률']
rate_of_change = rate_of_change.loc[:, rate_of_change.columns != 'type']
rate_of_change.reset_index(drop=True, inplace=True)
rate_of_change.columns = ['date', 'national_rate_of_change', 'seoul_rate_of_change']
rate_of_change['date'] = jeonse_price_index['date']

# 데이터프레임 병합
df_jeonse_price_index = pd.merge(jeonse_price_index, rate_of_change, on='date')
df_jeonse_price_index = df_jeonse_price_index[['date', 'national_jeonse_price_index', 'national_rate_of_change', 'seoul_jeonse_price_index', 'seoul_rate_of_change']]

# date 형식 변환
df_jeonse_price_index['date'] = pd.to_datetime(df_jeonse_price_index['date'], format="%Y년 %m월")

# 데이터프레임을 엑셀 파일로 저장
df_jeonse_price_index.to_excel("../data/jeonse_price_idex.xlsx", index=False, engine='openpyxl')


print(df_jeonse_price_index)

          date national_jeonse_price_index national_rate_of_change  \
0   2003-11-01                       49.74                    0.00   
1   2003-12-01                       49.24                   -1.00   
2   2004-01-01                       49.08                   -0.32   
3   2004-02-01                       49.32                    0.48   
4   2004-03-01                       49.59                    0.56   
..         ...                         ...                     ...   
245 2024-04-01                       89.03                    0.12   
246 2024-05-01                       89.20                    0.19   
247 2024-06-01                       89.35                    0.17   
248 2024-07-01                       89.57                    0.24   
249 2024-08-01                       89.85                    0.31   

    seoul_jeonse_price_index seoul_rate_of_change  
0                      46.74                 0.00  
1                      46.22                -1.11  
2  

In [11]:
import mysql.connector

conn = mysql.connector.connect(
    host = "eda-database.c3oek28e8yta.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "whskdhs",
    password = "whskdhs1234",
    database = "workspace"
)

cursor = conn.cursor(buffered=True)

df = pd.read_excel('../data/jeonse_price_idex.xlsx')

In [13]:
# 데이터베이스에 저장
def insert_data_to_jeonse_price_index(conn, cursor, df):
    try:
        for i, row, in df.iterrows():
            sql_jeonse = "INSERT INTO jeonse_price_index VALUES (%s, %s, %s, %s, %s)"
            values_jeonse = (row['date'], row['national_jeonse_price_index'], row['national_rate_of_change'], row['seoul_jeonse_price_index'], row['seoul_rate_of_change'])
            cursor.execute(sql_jeonse, values_jeonse)
            print(f"Inserted row {i+1} into article: {row['date']}")
            conn.commit()
    
    except mysql.connector.Error as error:
        print(f"Error inserting data: {error}")

In [14]:
insert_data_to_jeonse_price_index(conn, cursor, df)

Inserted row 1 into article: 2003-11-01 00:00:00
Inserted row 2 into article: 2003-12-01 00:00:00
Inserted row 3 into article: 2004-01-01 00:00:00
Inserted row 4 into article: 2004-02-01 00:00:00
Inserted row 5 into article: 2004-03-01 00:00:00
Inserted row 6 into article: 2004-04-01 00:00:00
Inserted row 7 into article: 2004-05-01 00:00:00
Inserted row 8 into article: 2004-06-01 00:00:00
Inserted row 9 into article: 2004-07-01 00:00:00
Inserted row 10 into article: 2004-08-01 00:00:00
Inserted row 11 into article: 2004-09-01 00:00:00
Inserted row 12 into article: 2004-10-01 00:00:00
Inserted row 13 into article: 2004-11-01 00:00:00
Inserted row 14 into article: 2004-12-01 00:00:00
Inserted row 15 into article: 2005-01-01 00:00:00
Inserted row 16 into article: 2005-02-01 00:00:00
Inserted row 17 into article: 2005-03-01 00:00:00
Inserted row 18 into article: 2005-04-01 00:00:00
Inserted row 19 into article: 2005-05-01 00:00:00
Inserted row 20 into article: 2005-06-01 00:00:00
Inserted 